In [ ]:
import datetime as dt
import json

import requests
from flask import Flask, jsonify, request

# create your API token, and set it up in Postman collection as part of the Body section
API_TOKEN = “1234”
# you can get API keys for free here - https://api-docs.pgamerx.com/
RSA_API_KEY = "f0bc32ec477c452bb29201150232002"

app = Flask(__name__)


def get_weather(location: str, date_time: str):
    url_base_url = "http://api.weatherapi.com/v1/forecast.json?key=f0bc32ec477c452bb29201150232002&q=07112&days=7"
url_endpoint = “v1”
 query_param = {
		“place_id” : get_place_id(location),
		“date” : date_time,
		“units” : “metric”
}

    payload = {}
    headers = {"Authorization": RSA_API_KEY}

    response = requests.request("GET", url, headers=headers, data=payload, params=query_params)
    return json.loads(response.text)


class InvalidUsage(Exception):
    status_code = 400

    def __init__(self, message, status_code=None, payload=None):
        Exception.__init__(self)
        self.message = message
        if status_code is not None:
            self.status_code = status_code
        self.payload = payload

    def to_dict(self):
        rv = dict(self.payload or ())
        rv["message"] = self.message
        return rv


@app.errorhandler(InvalidUsage)
def handle_invalid_usage(error):
    response = jsonify(error.to_dict())
    response.status_code = error.status_code
    return response


@app.route("/")
def home_page():
    return "<p><h2>KMA L2: Python Saas.</h2></p>"


@app.route(
    "/content/api/v1/integration/generate",
    methods=["POST"],
)
def joke_endpoint():
    start_dt = dt.datetime.now()
    json_data = request.get_json()

    if json_data.get("token") is None:
        raise InvalidUsage("token is required", status_code=400)

    token = json_data.get("token")

    if token != API_TOKEN:
        raise InvalidUsage("wrong API token", status_code=403)

    exclude = ""
    if json_data.get("exclude"):
        exclude = json_data.get("exclude")

    weather = get_weather(location: str, date_time: str)

    end_dt = dt.datetime.now()

    result = {
        “place_id” : get_place_id(location),
		“date” : date_time,
		“units” : “metric”
        "weather: weather,
    }

    return result